In [6]:
import pandas as pd
df=pd.read_csv('/Users/aniket/Documents/RAI/rating.csv')

In [16]:
d={}
df.sort_values("movieId")
data=df.iloc[:1000000]
data['rating']=data['rating']/5
for i in data.index:
    # print(data['movieId'][i])

    if data['movieId'][i] not in d.keys():
        d[data['movieId'][i]]=[data['rating'][i]]
    else:
        d[data['movieId'][i]].append(data['rating'][i])

data.head()

/var/folders/_1/snbz0_7d71v_hvnn4cb8zyz80000gn/T/ipykernel_15366/2336906855.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['rating']=data['rating']/5


,userId,movieId,rating,timestamp
0,1,2,0.7,2005-04-02 23:53:47
1,1,29,0.7,2005-04-02 23:31:16
2,1,32,0.7,2005-04-02 23:33:39
3,1,47,0.7,2005-04-02 23:32:07
4,1,50,0.7,2005-04-02 23:29:40


In [17]:
# print(d)
# data=data.drop(columns='timestamp')
data_to_use={}
for i in d.keys():
    if len(d[i])>500:
        data_to_use[i]=d[i]
print(len(d))
print(len(data_to_use))
# print(data_to_use)

# data_grouped=data.groupby('movieId')

13950
467


In [18]:
import numpy as np

true_mean={}
for i in data_to_use.keys():
    true_mean[i]=np.average(data_to_use[i])
print(true_mean)

{2: 0.6528352835283527, 32: 0.780952380952381, 47: 0.8152255639097745, 50: 0.874331777683496, 112: 0.6744932432432431, 151: 0.7046268656716418, 223: 0.7743073047858943, 253: 0.6989169675090253, 260: 0.8373396848662513, 293: 0.8135239567233385, 296: 0.8298795180722891, 318: 0.8939888925187847, 337: 0.7396825396825396, 367: 0.637406340057637, 541: 0.831630648330059, 589: 0.7894634525660964, 593: 0.8359415584415584, 653: 0.6528301886792452, 919: 0.7886518771331058, 924: 0.7886507936507936, 1036: 0.7881528662420383, 1079: 0.7767137096774194, 1080: 0.7929389312977099, 1089: 0.8112443778110945, 1090: 0.7834183673469388, 1097: 0.7509363295880149, 1136: 0.8358974358974359, 1193: 0.8456807187284037, 1196: 0.8318379790940766, 1198: 0.8394314534617148, 1200: 0.7968840579710144, 1201: 0.8291262135922332, 1208: 0.8188841201716738, 1214: 0.8008365508365508, 1215: 0.7544554455445543, 1219: 0.8225937183383992, 1222: 0.810126582278481, 1240: 0.7825499412455935, 1246: 0.7787584869059165, 1258: 0.7984654

In [23]:
import random
def pull(key):
    return random.choice(data_to_use[key])

In [24]:
import math
import numpy as np

# def pull(mean):
#     if mean==-1:
#         return np.random.binomial(1, 0.5)
#     return np.random.binomial(1, mean)

# def pull(mean):
#     return np.random.normal(mean, 1)


def epsilon(R):
    return (2**(-R))/4

def delta_R(R,delta,N):
    return (6*delta)/(((3.14*R)**2)*N)

def T_R(R,delta, N):
  epsilon_R = (2**(-R))/4
  return (2/epsilon_R**2)*np.log((R**2 * N * np.pi**2)/(3*delta))


def check_true(required,cluster,Arms,c,pulls,v):
    count=0
    # if v>0 and c>1.3*(pulls/v):
    #     return False
    for i in Arms:
        if i[1]==-1:
            count+=1
    if count==len(Arms):
        return False
    for i in required:
        if i>0:
            return True
    return False

def Update_cluster(required,cluster):
    required_new,cluster_new=[],[]
    r=0
    c=0
    t=1
    t_cluster=0
    for i in range(len(required)-1):
        if required[i]==0 and t==1:
            t_cluster+=cluster[i]
        elif required[i]==0 and t==0:
              
            cluster[i+1]+=cluster[i]
            
        elif required[i]!=0 and t==1:
            t=0
            required_new.append(0)
            cluster_new.append(t_cluster) 
            required_new.append(required[i])
            cluster_new.append(cluster[i])

        else:
            t=0
            required_new.append(required[i])
            cluster_new.append(cluster[i])
    required_new.append(required[len(required)-1])
    cluster_new.append(cluster[len(cluster)-1])    
    return required_new,cluster_new        

def remove_common(a, b):
  for i in a[:]:
      if i in b:
          a.remove(i)
  return a

def update(cluster,required,R,u_cap):
    t=0
    j=0
    while(j<len(u_cap) and u_cap[j][1]==-1):
        j+=1
    i=0
    upper=0
    if t>=len(cluster) or (cluster[t]+j+i-1)>=len(u_cap):
        upper=math.inf
    else:
        upper=u_cap[cluster[t]+j+i-1][0]

    lower=-math.inf

    for i in range(j,len(u_cap)):
        if u_cap[i][1]!=-1:
            if t<len(cluster) and i==cluster[t]+j:
                t+=1
                lower=u_cap[i-1][0]
                if t>=len(cluster) or (cluster[t]+j+i-1)>=len(u_cap):
                    upper=math.inf
                else:
                    upper=u_cap[cluster[t]+j+i-1][0]
            if (u_cap[i][0]>=lower+epsilon(R) or lower==u_cap[i][0]) and (u_cap[i][0]<=upper-epsilon(R) or u_cap[i][0]==upper):
                u_cap[i][1]=-1
                if t<len(cluster):
                    required[t]-=1
                    cluster[t]-=1
    return u_cap,required,cluster

def update2(cluster,required,R,u_cap):
    j=0
    t=0
    c=0
    cluster_temp=[]
    upper=math.inf
    lower=u_cap[cluster[j]][0]
    # print(required)
    # print(cluster)
    for i in range(len(u_cap)):
        if j<len(cluster) and i<cluster[j]+t:
            if u_cap[i][0]-lower>epsilon(R) and upper-u_cap[i][0]>epsilon(R) and u_cap[i][1]!=-1:
                u_cap[i][1]=-1
                
                c+=1
        elif j<len(cluster) and i==cluster[j]+t:
            if t+cluster[j]==0:
                upper==math.inf
            else:    
                upper=u_cap[t+cluster[j]-1][0]
            t+=cluster[j]
            if (j+1)<len(cluster) and t+cluster[j+1]<len(u_cap):
                lower=u_cap[t+cluster[j+1]][0]
            else:
                lower=-math.inf      
            j+=1
            cluster_temp.append(c)
            c=0
            if u_cap[i][0]-lower>epsilon(R) and upper-u_cap[i][0]>epsilon(R) and u_cap[i][1]!=-1:
                u_cap[i][1]=-1
    
                c+=1
    if len(cluster_temp)<len(cluster):
        cluster_temp.append(c)
    for i in range(len(cluster)):
        required[i]-=cluster_temp[i]

    return u_cap,required,cluster


def smart_algo(delta,cluster,required,Arms,merging,pulls,v):
    u_cap=[[pull(i),1,i,1] for i in Arms]
    u_cap.sort(key=lambda x:x[2],reverse=True)
    count=len(Arms)
    R=0
    while(check_true(required,cluster,u_cap,count,pulls,v)):
        if merging==True:
            required,cluster=Update_cluster(required,cluster)
    
        R+=1
        num_pulls=T_R(R,delta,len(Arms))
        if R>1:
            num_previous=T_R(R-1,delta,len(Arms))
            to_pull=int(num_pulls)-int(num_previous)
        else:
            num_previous=0
            to_pull=int(T_R(1,delta,len(Arms)))
        for _ in range(to_pull):
            for i in range(len(Arms)):
                
                if u_cap[i][1]!=-1:
                    reward=pull(u_cap[i][2])
                    u_cap[i][0]=(u_cap[i][0]*u_cap[i][3]+reward)/(u_cap[i][3]+1)
                    u_cap[i][3]+=1
                    count+=1
            
        u_cap.sort(key=lambda x:x[0],reverse=True)
        u_cap,required, cluster=update2(cluster,required,R,u_cap)
    # print(u_cap)

    u_cap,required, cluster=update2(cluster,required,R,u_cap)
    return count


def butterscotch(Arms,cluster,required,delta,merging,pulls,v):
    return smart_algo(delta,cluster,required,Arms,merging,pulls,v)


In [26]:
import math
import numpy as np

# def pull(mean):
#     return np.random.normal(mean, 1)

# def pull(mean):
#     if mean==-1:
#         return np.random.binomial(1, 0.5)
#     return np.random.binomial(1, mean)

def alpha(R,delta,N):
    return math.sqrt(math.log(((math.pi**2)*(R**2)*N)/(3*delta))/(2*R))

def check_true(required,cluster,u_cap,c,pulls,v):
    count=0
    # if v>0 and c>1.5*(pulls/v):
    #     return False
    for i in u_cap:
        if i[1]==-1:
            count+=1
    if count==len(u_cap):
        return False
    count=0
    for i in required:
        if i>0:
            return True
    return False

def Update_cluster(required,cluster):
    required_new,cluster_new=[],[]
    r=0
    c=0
    t=1
    t_cluster=0
    for i in range(len(required)-1):
        if required[i]==0 and t==1:
            t_cluster+=cluster[i]
        elif required[i]==0 and t==0:
              
            cluster[i+1]+=cluster[i]
        elif required[i]!=0 and t==1:
            t=0
            required_new.append(0)
            cluster_new.append(t_cluster) 
            required_new.append(required[i])
            cluster_new.append(cluster[i])

        else:
            t=0
            required_new.append(required[i])
            cluster_new.append(cluster[i])
    required_new.append(required[len(required)-1])
    cluster_new.append(cluster[len(cluster)-1])    
    return required_new,cluster_new        


def update(cluster,required,R,u_cap):
    t=0
    j=0
    while(j<len(u_cap) and u_cap[j][1]==-1):
        j+=1
    i=0
    upper=0
    if t>=len(cluster) or (cluster[t]+j+i-1)>=len(u_cap):
        upper=math.inf
    else:
        upper=u_cap[cluster[t]+j+i-1][0]
    lower=-math.inf

    for i in range(j,len(u_cap)):
        # if u_cap[i][1]!=-1:
        if t<len(cluster) and i==cluster[t]+j:
            t+=1
            lower=u_cap[i-1][0]
            if t>=len(cluster) or (cluster[t]+j+i-1)>=len(u_cap):
                upper=math.inf
            else:
                upper=u_cap[cluster[t]+j+i-1][0]
        if (u_cap[i][0]>=lower+alpha(R,delta,len(u_cap)) or lower==u_cap[i][0]) and (u_cap[i][0]<=upper-alpha(R,delta,len(u_cap)) or u_cap[i][0]==upper):
            u_cap[i][1]=-1
            if t<len(cluster):
                required[t]-=1
                cluster[t]-=1
    return u_cap,required,cluster

def update2(cluster,required,R,u_cap,delta):
    j=0
    t=0
    c=0
    cluster_temp=[]
    upper=math.inf
    lower=u_cap[cluster[j]][0]
    
    for i in range(len(u_cap)):
        if j<len(cluster) and i<cluster[j]+t:
            if u_cap[i][0]-lower>=2*alpha(R,delta,len(u_cap)) and upper-u_cap[i][0]>=2*alpha(R,delta,len(u_cap)) and u_cap[i][1]!=-1:
                u_cap[i][1]=-1
                c+=1
        elif j<len(cluster) and i==cluster[j]+t:
            if t+cluster[j]==0:
                upper==math.inf
            else:    
                upper=u_cap[t+cluster[j]-1][0]
            t+=cluster[j]
            if (j+1)<len(cluster) and t+cluster[j+1]<len(u_cap):
                lower=u_cap[t+cluster[j+1]][0]
            else:
                lower=-math.inf      
            j+=1
            cluster_temp.append(c)
            c=0
            if u_cap[i][0]-lower>=2*alpha(R,delta,len(u_cap)) and upper-u_cap[i][0]>=2*alpha(R,delta,len(u_cap)) and u_cap[i][1]!=-1:
                u_cap[i][1]=-1
                c+=1
    if len(cluster_temp)<len(cluster):
        cluster_temp.append(c)
    for i in range(len(cluster)):
        required[i]-=cluster_temp[i]

    return u_cap,required,cluster


def update_u_cap(R,mean,delta,N,u_cap):
    u_now=pull(mean)
    return (u_now+u_cap*(R-1))/(R),1

def vanila_algo(delta,cluster,required,Arms,merging,pulls,v):

    u_cap=[[pull(i),i] for i in Arms]
    count=len(Arms)
    R=0
   
    while(check_true(required,cluster,u_cap,count,pulls,v)):
        if merging==True:
            required,cluster=Update_cluster(required,cluster)
   
        u_cap.sort(key=lambda x:x[0],reverse=True)
        R+=1
        for i in range(len(Arms)):
            if u_cap[i][1]!=-1:
                u_cap[i][0],c=update_u_cap(R,u_cap[i][1],delta,len(Arms),u_cap[i][0])
                count+=c
        u_cap,required, cluster=update2(cluster,required,R,u_cap,delta)
    
    return count

def vanilla(Arms,cluster,required,delta,merging,pulls,v):
    return vanila_algo(delta,cluster,required,Arms,merging,pulls,v)


In [29]:
#Working on it not complete

import math
import numpy as np
from Butterscotch import smart_algo as algo

# np.random.seed(10)

# def pull(mean):
#     return np.random.normal(mean, 1)

# def pull(mean):
#     if mean==-1:
#         return np.random.binomial(1, 0.5)
#     return np.random.binomial(1, mean)

def alpha(R,delta,N,R_i):
    return math.sqrt(math.log(((math.pi**2)*(R**3)*N)/(3*delta))/(2*R_i))

# def alpha(R,delta,N,R_i):
#     return math.sqrt(math.log((5*(R**4)*N)/(4*delta))/(2*R_i))

def check_true(required,cluster,u_cap,c,pulls,v):
    count=0
    # if v>0 and c>1.3*(pulls/v):
    #     return False
    
    for i in u_cap:
        if i[5]==-1:
            count+=1
    if count==len(u_cap):
        return False
    for i in required:
        if i>0:
            return True
    return False
# def check_true(required,cluster,Arms,c,pulls,v):
#     count=0
#     if v>0 and c>1.3*(pulls/v):
#         return False
    
#     for i in Arms:
#         if i[5]==-1:
#             count+=1
#     # print("count-----",count)        
#     if count==len(Arms):
#         return False
#     for i in required:
#         if i>0:
#             return True
#     return False

def update(cluster,required,R,u_cap):
    t=0
    j=0
    while(j<len(u_cap) and u_cap[j][1]==-1):
        j+=1
    i=0
    upper=0
    if t>=len(cluster) or (cluster[t]+j+i-1)>=len(u_cap):
        upper=math.inf
    else:
        upper=u_cap[cluster[t]+j+i-1][0]
    lower=-math.inf

    for i in range(j,len(u_cap)):
        if u_cap[i][1]!=-1:
            if t<len(cluster) and i==cluster[t]+j:
                t+=1
                lower=u_cap[i-1][0]+alpha(u_cap[i-1][4],delta,len(u_cap))
                if t>=len(cluster) or (cluster[t]+j+i-1)>=len(u_cap):
                    upper=math.inf
                else:
                    upper=u_cap[cluster[t]+j+i-1][0]-alpha(u_cap[i-1][4],delta,len(u_cap))
            if (u_cap[i][0]>=lower+alpha(u_cap[i][4],delta,len(u_cap)) or lower==u_cap[i][0]) and (u_cap[i][0]<=upper-alpha(u_cap[i][4],delta,len(u_cap)) or u_cap[i][0]==upper):
                u_cap[i][1]=-1
                if t<len(cluster):
                    required[t]-=1
                    cluster[t]-=1
    return u_cap,required,cluster

def update2(cluster,required,R,u_cap,delta):
    j=0
    t=0
    c=0
    cluster_temp=[]
    upper=math.inf
    lower=u_cap[cluster[j]][1]+alpha(R,delta,len(u_cap),u_cap[cluster[j]][4])
    
    for i in range(len(u_cap)):
        # print(i,c)
        # print("lower----",lower)
        # print("upper----",upper)
        # print("Arms----",u_cap[i])
        if j<len(cluster) and i<cluster[j]+t:
            if (u_cap[i][0]-alpha(R,delta,len(u_cap),u_cap[i][4]))>=lower and (u_cap[i][0]+alpha(R,delta,len(u_cap),u_cap[i][4]))<=upper and u_cap[i][5]!=-1:
                # print(u_cap[i][1])
                u_cap[i][5]=-1
                c+=1
        elif j<len(cluster) and i==cluster[j]+t:
            if t+cluster[j]==0:
                upper==math.inf
            else:    
                upper=u_cap[t+cluster[j]-1][0]-alpha(R,delta,len(u_cap),u_cap[t+cluster[j]-1][4])
            t+=cluster[j]
            if (j+1)<len(cluster) and t+cluster[j+1]<len(u_cap):
                lower=u_cap[t+cluster[j+1]][0] +alpha(R,delta,len(u_cap),u_cap[t+cluster[j+1]][4])
            else:
                lower=-math.inf      
            j+=1
            cluster_temp.append(c)
            c=0
            if (u_cap[i][0]-alpha(R,delta,len(u_cap),u_cap[i][4]))>=lower and (u_cap[i][0]+alpha(R,delta,len(u_cap),u_cap[i][4]))<=upper and u_cap[i][5]!=-1:
                # print(u_cap[i][1])
                u_cap[i][5]=-1
                c+=1    
    if len(cluster_temp)<len(cluster):
        cluster_temp.append(c)
    for i in range(len(cluster)):
        # cluster[i]-=cluster_temp[i]
        required[i]-=cluster_temp[i]
    # print("required------",required)
    return u_cap,required,cluster



def pull_arms(u_cap,cluster,delta,N,R):
    count=0
    done={}
    start=[]
    end=[]
    t=0
    for i in range(len(cluster)):
        start.append(t)
        t+=cluster[i]
        end.append(t)

    for i in range(len(cluster)):
        arr=u_cap[start[i]:end[i]]
        lcb=sorted(arr,key=lambda x:x[1],reverse=True)
        ucb=sorted(arr,key=lambda x:x[2],reverse=True)
        if lcb[-1][5]!=0:
            temp=(pull(lcb[-1][3])+lcb[-1][0]*lcb[-1][4])/(lcb[-1][4]+1)
            count+=1
            if lcb[-1][6] not in done.keys():
                done[lcb[-1][6]]=[temp,temp-alpha(R,delta,N,lcb[-1][4]),temp+alpha(R,delta,N,lcb[-1][4]),lcb[-1][3],lcb[-1][4]+1,lcb[-1][5],lcb[-1][6]]
        if ucb[0][5]!=0:
            temp=(pull(ucb[0][3])+ucb[0][0]*ucb[0][4])/(ucb[0][4]+1)
            count+=1
            if ucb[0][6] not in done.keys():
                done[ucb[0][6]]=[temp,temp-alpha(R,delta,N,ucb[0][4]),temp+alpha(R,delta,N,ucb[0][4]),ucb[0][3],ucb[0][4]+1,ucb[0][5],ucb[0][6]]
        
    for i in range(len(cluster)):
        temp=math.inf 
        ind=-1        
        for j in range(start[i],end[i]):
            if u_cap[j][5]!=0:
                if start[i]==0:
                    if temp<abs(u_cap[end[i]][0]-u_cap[j][0]):
                        temp=abs(u_cap[end[i]][0]-u_cap[j][0])
                        ind=j
                elif end[i]==len(Arms):
                    if temp<abs(u_cap[start[i]-1][0]-u_cap[j][0]):
                        temp=abs(u_cap[start[i]-1][0]-u_cap[j][0])
                        ind=j
                else:
                    if temp<min(abs(u_cap[start[i]-1][0]-u_cap[j][0]),abs(u_cap[end[i]][0]-u_cap[j][0])):
                        temp=min(abs(u_cap[start[i]-1][0]-u_cap[j][0]),abs(u_cap[end[i]][0]-u_cap[j][0]))
                        ind=j
        if ind!=-1:
            p=(pull(u_cap[ind][3])+u_cap[ind][0]*u_cap[ind][4])/(u_cap[ind][4]+1)
            count+=1
            if u_cap[ind][6] not in done.keys():   
                done[u_cap[ind][6]]=[p,p-alpha(R,delta,N,u_cap[ind][4]),p+alpha(R,delta,N,u_cap[ind][4]),u_cap[ind][3],u_cap[ind][4]+1,u_cap[ind][5],u_cap[ind][6]]
        
    
    for i in range(len(u_cap)):
        if u_cap[i][6] in done.keys():
            u_cap[i]=done[u_cap[i][6]]

    return u_cap,count

def update_u_cap(R,mean,delta,N,u_cap):
    u_now=pull(mean)
    return (u_now+u_cap*(R-1))/(R),1

def lucb_algo(delta,cluster,required,Arms,pulls,v):
    # Arms.sort
    pull_val=[[pull(i),i] for i in Arms]
    u_cap=[[i[0],i[0]-alpha(1,delta,len(Arms),1),i[0]+alpha(1,delta,len(Arms),1),i[1],1,1] for i in pull_val]
    u_cap.sort(key=lambda x:x[3],reverse=True)
    for i in range(len(Arms)) :
        u_cap[i].append(i)

    #first value=u_cap, second value=lcb, third value=ucb, forth value=true mean, fifth=no of pulls of arm i
    count=len(Arms)
    R=1

    while(check_true(required,cluster,u_cap,count,pulls,v)):
        # print(u_cap)
        u_cap.sort(key=lambda x:x[0],reverse=True)
        R+=1
        # print("--------------------------------",R)
        u_cap,c=pull_arms(u_cap,cluster,delta,len(Arms),R)
        count+=c
        u_cap,required, cluster=update2(cluster,required,R,u_cap,delta)
    u_cap,required, cluster=update2(cluster,required,R,u_cap,delta)
    # print(u_cap)
    t=0
    for i in u_cap:
        t+=i[4]
    return count


In [30]:
import itertools
N=10
data_used=dict(itertools.islice(data_to_use.items(), N))
cluster=[1,9]
required=[1,0]
delta=0.01
merging=True
pulls=10000
v=-1
print(butterscotch(data_used,cluster,required,delta,merging,pulls,v))
print(vanilla(data_used,cluster,required,delta,merging,pulls,v))
print(lucb_algo(delta,cluster,required,data_used,pulls,v))

IndexError: list index out of range

In [2]:
import pandas as pd
import numpy as np
import itertools
from ml_Butterscotch import butterscotch
from ml_vanilla import vanilla
from ml_lucb import lucb_algo

df=pd.read_csv('/Users/aniket/Documents/RAI/rating.csv')
print("---------------dataset loaded successfully----------------")
d={}
df.sort_values("movieId")
data=df.iloc[:1000000]
data['rating']=data['rating']/5
for i in data.index:
    # print(data['movieId'][i])

    if data['movieId'][i] not in d.keys():
        d[data['movieId'][i]]=[data['rating'][i]]
    else:
        d[data['movieId'][i]].append(data['rating'][i])

# data.head()
data_to_use={}
for i in d.keys():
    if len(d[i])>500:
        data_to_use[i]=d[i]
# print(len(d))
# print("Length of total data-------",len(data_to_use))

true_mean={}

for i in data_to_use.keys():
    true_mean[i]=np.average(data_to_use[i])

# print("Dictionary of True Means: ",true_mean)
# print("----------------------------------------------------------------")
true_mean={}

data_used=dict(itertools.islice(data_to_use.items(), 50))
print("data Used-------------\n",data_used)
# print("---------------------------------------------\n\n\n")
for i in data_used.keys():
    true_mean[i]=np.average(data_to_use[i])

# print("Dictionary of True Means: ",true_mean)
import json
with open('/Users/aniket/Documents/RAI/true_mean.json', 'w') as fp:
    json.dump(data_used, fp)
with open('/Users/aniket/Documents/RAI/avg_true_mean.json', 'w') as fp:
    json.dump(true_mean, fp)
pulls_butter=0
pulls_vanilla=0
pulls_lucb=0
# for i in range(10):
#     N=10
#     data_used=dict(itertools.islice(data_to_use.items(), N))
#     cluster=[3,5,2]
#     required=[3,5,2]
#     delta=0.01
#     merging=True
#     pulls=10000
#     v=-1
#     pulls_butter+=butterscotch(data_used,cluster,required,delta,merging,pulls,v)
#     pulls_vanilla+=vanilla(data_used,cluster,required,delta,merging,pulls,v)
#     pulls_lucb+=lucb_algo(delta,cluster,required,data_used,pulls,v)
#     print(pulls_butter)
#     print(pulls_vanilla)
#     print(pulls_lucb)
#     print()
# print("----------------------------------------------------------------")
# print("No of pulls------------",pulls_butter/10)
# print("No of pulls------------",pulls_vanilla/10)
# print("No of pulls------------",pulls_lucb/10)

    # N=20
    # data_used=dict(itertools.islice(data_to_use.items(), N))
    # cluster=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
    # required=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
    # delta=0.01
    # merging=True
    # pulls=10000
    # v=-1
    # print("----------------------------------------------------------------")
    # # print("No of pulls------------",butterscotch(data_used,cluster,required,delta,merging,pulls,v))
    # # print("No of pulls------------",vanilla(data_used,cluster,required,delta,merging,pulls,v))
    # # print("No of pulls------------",lucb_algo(delta,cluster,required,data_used,pulls,v))

    # N=30
    # data_used=dict(itertools.islice(data_to_use.items(), N))
    # cluster=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
    # required=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
    # delta=0.01
    # merging=True
    # pulls=10000
    # v=-1
    # print("----------------------------------------------------------------")
    # print("No of pulls------------",butterscotch(data_used,cluster,required,delta,merging,pulls,v))
    # print("No of pulls------------",vanilla(data_used,cluster,required,delta,merging,pulls,v))
    # # print("No of pulls------------",lucb_algo(delta,cluster,required,data_used,pulls,v))

    # N=40
    # data_used=dict(itertools.islice(data_to_use.items(), N))
    # cluster=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
    # required=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
    # delta=0.01
    # merging=True
    # pulls=10000
    # v=-1
    # print("----------------------------------------------------------------")
    # print("No of pulls------------",butterscotch(data_used,cluster,required,delta,merging,pulls,v))
    # print("No of pulls------------",vanilla(data_used,cluster,required,delta,merging,pulls,v))
    # # print("No of pulls------------",lucb_algo(delta,cluster,required,data_used,pulls,v))

    # N=50
    # data_used=dict(itertools.islice(data_to_use.items(), N))
    # cluster=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
    # required=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
    # delta=0.01
    # merging=True
    # pulls=10000
    # v=-1
    # print("----------------------------------------------------------------")
    # print("No of pulls------------",butterscotch(data_used,cluster,required,delta,merging,pulls,v))
    # print("No of pulls------------",vanilla(data_used,cluster,required,delta,merging,pulls,v))
    # # print("No of pulls------------",lucb_algo(delta,cluster,required,data_used,pulls,v))


---------------dataset loaded successfully----------------


/var/folders/_1/snbz0_7d71v_hvnn4cb8zyz80000gn/T/ipykernel_43044/1470682306.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['rating']=data['rating']/5


data Used-------------
 {2: [0.7, 0.6, 0.6, 0.6, 0.6, 0.6, 0.2, 0.7, 0.4, 0.8, 0.2, 0.4, 0.6, 0.6, 0.6, 0.2, 0.6, 0.6, 0.8, 0.6, 1.0, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.7, 0.6, 0.6, 1.0, 0.8, 1.0, 1.0, 0.5, 0.7, 0.9, 0.6, 1.0, 0.6, 0.8, 0.8, 0.6, 0.6, 0.4, 0.6, 0.4, 0.4, 0.8, 0.8, 1.0, 0.3, 0.6, 0.8, 0.8, 0.8, 0.5, 0.6, 0.1, 0.6, 0.7, 0.6, 0.8, 1.0, 0.6, 0.6, 0.7, 0.6, 1.0, 0.8, 0.7, 0.6, 0.6, 0.7, 0.5, 0.6, 0.6, 0.6, 1.0, 0.8, 0.8, 0.6, 0.6, 0.6, 0.6, 0.7, 0.8, 0.3, 0.8, 0.6, 0.4, 0.7, 1.0, 0.8, 0.4, 1.0, 0.9, 0.8, 0.7, 0.6, 0.4, 0.3, 0.4, 0.8, 0.8, 0.8, 0.8, 0.4, 0.6, 0.5, 0.3, 1.0, 0.4, 1.0, 0.6, 0.8, 0.7, 1.0, 0.3, 1.0, 0.6, 0.8, 0.5, 0.6, 0.6, 0.6, 0.6, 0.6, 0.8, 0.6, 0.4, 0.5, 0.8, 0.5, 0.7, 0.8, 0.1, 0.2, 0.8, 0.7, 0.6, 0.6, 0.6, 0.7, 0.8, 1.0, 0.6, 0.8, 0.8, 0.7, 0.6, 0.8, 0.8, 0.6, 0.6, 0.6, 0.7, 0.4, 0.8, 0.4, 0.6, 0.8, 0.8, 0.7, 0.1, 0.5, 0.6, 0.6, 0.6, 0.6, 0.6, 0.8, 0.3, 0.8, 0.8, 1.0, 0.6, 0.8, 0.8, 0.6, 1.0, 0.8, 0.8, 0.5, 0.3, 0.8, 0.7, 0.6, 0.6, 0.7, 0.6, 0.8, 0.8, 0.7, 0

TypeError: keys must be str, int, float, bool or None, not int64